In [1]:
# run this if you want to run the example from the examples folder without installing the package
import sys
sys.path.insert(0, "../")

In [2]:
# import the package
import osmosispy

ModuleNotFoundError: No module named 'google.protobuf.internal'

In [ ]:
# check the version
osmosispy.__version__

'14.0.2'

In [3]:
dir(osmosispy)

NameError: name 'osmosispy' is not defined